# MTH6412B: Projet voyageur de commerce (Phase 2)

*Florence Tanoi Namio*: **M1913387**

*Rémi Decouvelaere*: **M2035574**

## Objectif

L'objectif de la phase 2 est de pouvoir créer, à partir d'un graphe non orienté connexe existant, un arbre de recouvrement minimal.

### 1. Structure de données pour les composantes connexes

Durant l'exécution de l'algorithme de Kruskal, on construira progressivement un arbre de recouvrement minimal, en partant d'un graphe sans arête, et en lui ajoutant une nouvelle arête à chaque étape de manière à minimiser le coût tout en veillant à ne pas construire de cycles.

Ainsi, à chaque étape de l'algorithme, l'objet construit sera une forêt d'un ou plusieurs arbres. Il faudra alors pouvoir tester si deux noeuds de ce graphe appartienne au même ensemble connexe, c'est-à-dire au même arbre. 

Nous avons donc cherché à implémenter une structure de données qui pourrait représenter de manière efficace une forêt. Cette structure de données doit permettre d'identifier rapidement si deux noeuds sont dans le même ensemble connexe, et doit également permettre l'union de deux ensembles connexes différents.

#### Solution

Nous avons décidé d'orienter les arbres de façon à créer des arborescences. Ainsi, chaque sommet d'une arborescence sera relié par un chemin unique à la racine de l'arborescence, choisie arbitrairement. De cette façon, chaque sommet du graphe possèdera un unique parent.

On pourra alors représenter aisément une forêt par une **liste d'adjacence**, qui sera représenté par un dictionnaire, dans lequel on associera à chaque noeud du graphe son **unique** parent (*pour résoudre un bug, nous avons utilisé les noms des noeuds plutôt que les noeuds eux-même*). On ajoutera également la convention suivante afin d'identifier facilement les racines : la racine d'une arborescence est son propre parent.

L'initialisation du dictionaire correspond à l'initialisation de l'algorithme de Kruskal: le graphe de construction (futur arbre de recouvrement) est alors composé uniquement des points du graphe principal G. Ainsi, chaque point est son propre parent.

```julia
#Inialisation du dictionaire "parents", représentant les composantes connexes du graphe de construction
parents=Dict(name(node) => name(node) for node in nodes(G))
```

#### Union de deux arborescences

La fonction union!(edge,dict) permettra de relier deux arborescences de la forêt représentée par le dictionaire "dict" : il faudra veiller à ce que l'ensemble connexe obtenu soit bien une arborescence également.

Le principe de l'algorithme utilisé est le suivant: si le noeud1 de l'arête appartient à l'arborescence 1 et que le noeud2 appartient à l'arborescence 2, on va d'abord ajouter à l'arborescence 1 l'arête reliant noeud1 et noeud2, en l'orientant de noeud2 vers noeud1; puis on va remonter progressivement l'arborescence 2 jusqu'à sa racine, en réorientant une à une les arêtes de l'arborescence 2 vers la racine de l'arborescence 1. 
En pratique, il suffira seulement de réorienter les arcs du seul chemin reliant noeud2 à la racine de l'arborescence 2, car tous les autres arcs de l'arborescence 2 seront dirigés vers un des sommets traversés par ce chemin, et donc, lorsque ce chemin sera réorienté, vers la racine de l'arborescence 1.

```Julia
"""Fonction réalisant l'union de deux arborescences, par l'arête edge, et dans la forêt représentée par le dictionaire dict"""
function union!(edge::Edge{T},dict::Dict{String,String})
    #On récupère dans node1 et node2 les noms des deux noeuds contenus dans l'arête edge
    node1=name(data(edge)[1])
    node2=name(data(edge)[2])
    #p1 et p2 permettent de garder en mémoire l'arc reliant node2 et son parent
    p1=node2
    p2=dict[node2]
    dict[node2]=node1 #On remplace le parent de node2 par node1, c'est-à-dire qu'on supprime l'arc allant de p1 vers p2 et on le remplace vers un arc allant de p1 vers node1
    while p1!=p2 #Tant que le parent de p1 est différent de lui même (c'est-à-dire tant que l'on n'a pas atteint la racine de la deuxième arborescence)
        p3=dict[p2] #avant d'appliquer des changements au dictionaire, on mémorise le parent de p2
        dict[p2]=p1 #on peut alors appliquer les changements au dictionaire: l'arc reliant p1 et p2 change de sens, et l'arc reliant p2 et son ancient parent p3 disparait
        #on s'intéresse maintenant à l'arc que l'on vient d'effacer
        p1=p2
        p2=p3
    end
        #A la dernière itération, on aura effacé l'arc reliant l'ancienne racine de l'arborescence 2 à elle-même
end
```

#### Identification de la racine d'un noeud

La fonction root(node,dict) suivante permet de retracer dans une forêt d'arborescences (représentée par un dictionaire) la racine d'un noeud. Cette fonction permettra de savoir si deux noeuds sont dans le même ensemble connexe, car ils possèderont alors la même racine. Le principe de cette fonction et de remonter les ancêtres du noeud jusqu'à ce que l'un d'entre eux soit son propre parent : celui-là sera la racine.

```Julia
"""Fonction renvoyant la racine d'un noeud node dans une forêt d'arborescence représenté par un dictionaire dict"""
function root(node::Node{T},dict::Dict{String,String})
    p=name(node)
    #Tant que le parent du noeud p est différent de lui-même, on remplace ce noeud par son parent
    while p!=dict[p]
        p=dict[p]
    end
    return(p)
    end```

#### Test de connexité entre deux noeuds

La fonction connex(edge,dict) permettra de réaliser le test de connexité entre deux noeuds dans l'algorithme de Kruskal. Grâce à la structure de données implémentée pour représenter une forêt, il est assez simple de savoir si deux noeuds sont dans le même ensemble connexe: en effet, chaque arborescence de la forêt possède une unique racine. Il suffit donc de comparer les racines de deux noeuds pour savoir s'ils appartiennent à la même arborescence.

```Julia
"""Fonction renvoyant un booléen true si les deux noeuds de l'arête edge sont dans le même ensemble connexe et false autrement"""
function connex(edge::Edge{T},dict::Dict{String,String}) where T
    #On récupère dans node1 et node2 les deux noeuds contenus dans l'arête edge
    node1=data(edge)[1]
    node2=data(edge)[2]
    #Si ces deux noeuds ont la même racine, ils sont dans le même ensemble connexe, sinon non
    return(root(node1,dict)==root(node2,dict))
    end```

### 2. Implémentation de l'algorithme de Kruskal

Voici comment nous avons implémenté l'algorithme de Kruskal:
- Le graphe *G_construction* représente le futur arbre de recouvrement, à chaque étape de sa construction
- Le dictionaire *parents* représente la forêt d'arborescences construite dans *G_construction* afin d'identifier facilement ses composantes connexes
- Le vecteur d'arêtes E contient la liste d'arêtes de G, triées par coût croissant

La première étape de l'algorithme de Kruskal consiste à trier les arêtes:
```Julia
E = edges(G)
sort!(E, by = x -> weight(x))```

On itialise aussi le graphe de construction *G_construction* ainsi que la forêt d'arborescences qui lui est associée *parents*:
```Julia
parents=Dict(name(node) => name(node) for node in nodes(G))
#Graphe contenant les noeuds de G, initialement sans arêtes
G_construction=Graph("Arbre"), nodes(G), Edge{Vector{Float64}}[])[])```

L'étape suivante de l'algorithme est celle-ci: pour chaque arête, de la plus légère à la plus lourde:
- Vérifier si ses deux sommets sont dans le même ensemble connexe, dans *G_construction* (ce qui est réalisé avec la fonction **connex()**)
- Si ce n'est pas le cas, ajouter cette arête au graphe de construction *G_construction* et à la forêt d'arborescences associée *parents* (ce qui est réalisé avec la fonction **union()**)

```Julia
"""Fonction renvoyant un arbre de recouvrement de coût minimal associé au graphe G, en utilisant l'algorithme de Kruskal"""
function kruskal(G::Graph)
     E = edges(G)
     sort!(E, by = x -> weight(x)) #Tri des arêtes par poids
     parents=Dict(name(node) => name(node) for node in nodes(G))
     G_construction=Graph("Arbre", nodes(G), Edge{Vector{Float64}}[]) #Graphe contenant les noeuds de G, initialement sans arêtes
     for e in E
        if connex(e,parents)==false #Si les deux noeuds de l'arête e ne sont pas dans le même ensemble connexe
            add_edge!(G_construction,e) #On ajoute cette arête au graphe de construction
            union!(e,parents) #On ajoute cette arête à la forêt d'arborescence
        end
    end
    return G_construction #Le graphe de construction obtenu est un arbre de recouvrement de G
end```

### 3. Tests unitaires

Nous avons progressivement rédigé un fichier de tests unitaires, en utilisant un graphe assez simple, afin de pouvoir tester le fonctionnement de chacune des fonctions implémentées.

```Julia
using Test
#Initialisation du graphe test G
node1 = Node("1", [-1.0,0.0])
node2 = Node("2", [0.0,1.0])
node3 = Node("3", [1.0,0.0])
node4 = Node("4", [0.0,-1.0])
arete1 = Edge((node1, node2), 1)
arete2 = Edge((node2, node4), 2)
arete3 = Edge((node2, node3), 3)
arete4 = Edge((node1, node4), 4)
arete5 = Edge((node3, node4), 5)
G = Graph("Test graph", [node1, node2, node3, node4], [arete1, arete2, arete3, arete4, arete5])

parents=Dict(name(node) => name(node) for node in nodes(G)) #Création d'une forêt d'arborescences avec les noeuds de G, et dont chaque noeud est son propre parent

for node in nodes(G) #Vérification du contenu de parents
    @test parents[name(node)] == name(node)
end

for edge in edges(G) #On vérifie qu'aucun couple de sommets n'est dans le même ensemble connexe, dans parents
    @test connex(edge,parents) == false
end

union!(arete1,parents) #On ajoute l'arête 1 à parents
@test parents[name(node2)]==name(node1) #On vérifie que le parent de node2 est bien devenu node1
@test parents[name(node1)]==name(node1)
@test connex(arete1,parents) == true #on vérifie le fonctionnement de la fonction connex
@test connex(arete2,parents) == false
@test connex(arete3,parents) == false
@test connex(arete4,parents) == false
@test connex(arete5,parents) == false

union!(arete2,parents)
@test parents[name(node4)]==name(node2)
@test parents[name(node2)]==name(node1)
@test parents[name(node1)]==name(node1)
@test connex(arete1,parents) == true
@test connex(arete2,parents) == true
@test connex(arete3,parents) == false
@test connex(arete4,parents) == true
@test connex(arete5,parents) == false

#Vérification des racines de chaque noeud, dans le nouveau "parents"
@test root(node4,parents)==name(node1)
@test root(node2,parents)==name(node1)
@test root(node1,parents)==name(node1)
@test root(node3,parents)==name(node3)

union!(arete3,parents) #Ajout de la dernière arête possible

#Vérification des racines de chaque noeud, dans le nouveau "parents"
@test root(node4,parents)==name(node1)
@test root(node2,parents)==name(node1)
@test root(node1,parents)==name(node1)
@test root(node3,parents)==name(node1)

for edge in edges(G) #On vérifie que tous les couples de sommets sont dans le même ensemble connexe
    @test connex(edge,parents) == true
end


#Vérification du fonctionnement de la fonction Kruskal
K=kruskal(G)
typeof(K)==typeof(G)

total_weight = sum([weight(e) for e in edges(K)])

for e in edges(K)
@test connex(e,parents)==true #On vérifie que l'ensemble obtenu est connexe

end

@test total_weight==6 #On vérifie que l'arbre obtenu est bien de coût minimal

#On vérifie que K ne contient pas de cycles
@test !(arete4 in edges(K))
@test !(arete5 in edges(K))

```

### 4. Application

#### Code principal

Pour une utilisation facile de la fonction Kruskal, nous avons créé un programme principal qui permet d'obtenir, à partir du nom d'un fichier *tsp*, un arbre de recouvrement (objet de type **Graph**) associé au graphe correspondant.

La première partie du programme **main.jl** reprend le programme principal **main.jl** de la phase 1, c'est-à-dire qu'il crée un objet de type **Graph** à partir du nom d'un fichier *tsp*.

La deuxième partie du programme consiste simplement à appliquer la fonction **kruskal()** à ce graphe, afin de créer un arbre de recouvrement.

Enfin, nous avons rajouté une dernière partie qui consiste à afficher l'arbre de recouvrement obtenu, en réutilisant les fonctions de la phase 1.

```Julia
arbre=kruskal(G)
show(arbre)
tree_weight = sum([weight(e) for e in edges(arbre)])
println("Le poids de l'arbre de recouvrement est de ",tree_weight)
```

#### Test sur diverses instances de TSP symétriques

Nous avons testé notre code **main.jl** sur plusieurs instances de *tsp*, et le résultat obtenu semble toujours correct. Cependant, le temps d'exécution est parfois long (une dizaine de secondes).